In [1]:
import numpy as np
import scipy as sp
import scipy.stats
import itertools
import logging
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as utils
import math
import time
import tqdm

import torch
import torch.optim as optim
import torch.nn.functional as F
from argparse import ArgumentParser
from torch.distributions import MultivariateNormal

import torch.nn as nn
import torch.nn.init as init

from flows import RealNVP, Planar
from models import NormalizingFlowModel

## Load and process the data

In [2]:
f_PureBkg = pd.read_hdf("/data/t3home000/spark/LHCOlympics/data/MassRatio_pureBkg.h5")
dt_PureBkg = f_PureBkg.values

In [3]:
dt_PureBkg[:,1] = (dt_PureBkg[:,1]-np.mean(dt_PureBkg[:,1]))/np.std(dt_PureBkg[:,1])
dt_PureBkg[:,2] = (dt_PureBkg[:,2]-np.mean(dt_PureBkg[:,2]))/np.std(dt_PureBkg[:,2])
dt_PureBkg[:,3] = (dt_PureBkg[:,3]-np.mean(dt_PureBkg[:,3]))/np.std(dt_PureBkg[:,3])
dt_PureBkg[:,4] = (dt_PureBkg[:,4]-np.mean(dt_PureBkg[:,4]))/np.std(dt_PureBkg[:,4])
dt_PureBkg[:,5] = (dt_PureBkg[:,5]-np.mean(dt_PureBkg[:,5]))/np.std(dt_PureBkg[:,5])
dt_PureBkg[:,6] = (dt_PureBkg[:,6]-np.mean(dt_PureBkg[:,6]))/np.std(dt_PureBkg[:,6])


dt_PureBkg[:,8] = (dt_PureBkg[:,8]-np.mean(dt_PureBkg[:,8]))/np.std(dt_PureBkg[:,8])
dt_PureBkg[:,9] = (dt_PureBkg[:,9]-np.mean(dt_PureBkg[:,9]))/np.std(dt_PureBkg[:,9])
dt_PureBkg[:,10] = (dt_PureBkg[:,10]-np.mean(dt_PureBkg[:,10]))/np.std(dt_PureBkg[:,10])
dt_PureBkg[:,11] = (dt_PureBkg[:,11]-np.mean(dt_PureBkg[:,11]))/np.std(dt_PureBkg[:,11])
dt_PureBkg[:,12] = (dt_PureBkg[:,12]-np.mean(dt_PureBkg[:,12]))/np.std(dt_PureBkg[:,12])

dt_PureBkg[:,14] = (dt_PureBkg[:,14]-np.mean(dt_PureBkg[:,14]))/np.std(dt_PureBkg[:,14])
dt_PureBkg[:,15] = (dt_PureBkg[:,15]-np.mean(dt_PureBkg[:,15]))/np.std(dt_PureBkg[:,15])
dt_PureBkg[:,16] = (dt_PureBkg[:,16]-np.mean(dt_PureBkg[:,16]))/np.std(dt_PureBkg[:,16])
dt_PureBkg[:,17] = (dt_PureBkg[:,17]-np.mean(dt_PureBkg[:,17]))/np.std(dt_PureBkg[:,17])
dt_PureBkg[:,18] = (dt_PureBkg[:,18]-np.mean(dt_PureBkg[:,18]))/np.std(dt_PureBkg[:,18])
dt_PureBkg[:,19] = (dt_PureBkg[:,19]-np.mean(dt_PureBkg[:,19]))/np.std(dt_PureBkg[:,19])


dt_PureBkg[:,21] = (dt_PureBkg[:,21]-np.mean(dt_PureBkg[:,21]))/np.std(dt_PureBkg[:,21])
dt_PureBkg[:,22] = (dt_PureBkg[:,22]-np.mean(dt_PureBkg[:,22]))/np.std(dt_PureBkg[:,22])
dt_PureBkg[:,23] = (dt_PureBkg[:,23]-np.mean(dt_PureBkg[:,23]))/np.std(dt_PureBkg[:,23])
dt_PureBkg[:,24] = (dt_PureBkg[:,24]-np.mean(dt_PureBkg[:,24]))/np.std(dt_PureBkg[:,24])
dt_PureBkg[:,25] = (dt_PureBkg[:,25]-np.mean(dt_PureBkg[:,25]))/np.std(dt_PureBkg[:,25])

In [4]:
total_PureBkg = torch.tensor(dt_PureBkg)
total_PureBkg_train_x_1 = total_PureBkg.t()[1:7].t()
total_PureBkg_train_x_2 = total_PureBkg.t()[8:13].t()
total_PureBkg_train_x_3 = total_PureBkg.t()[14:20].t()
total_PureBkg_train_x_4 = total_PureBkg.t()[21:26].t()

total_PureBkg_selection = torch.cat((total_PureBkg_train_x_1,total_PureBkg_train_x_2,total_PureBkg_train_x_3,total_PureBkg_train_x_4),dim=1)

In [5]:
bs = 1000
bkgAE_train_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs, shuffle=True) 
bkgAE_test_iterator = utils.DataLoader(total_PureBkg_selection, batch_size=bs)

## Build the model

In [6]:
class VAE_NF(nn.Module):
    def __init__(self, K, D):
        super().__init__()
        self.dim = D
        self.K = K
        self.encoder = nn.Sequential(
            nn.Linear(22, 96),
            nn.LeakyReLU(True),
            nn.Linear(96, 48),
            nn.LeakyReLU(True),            
            nn.Linear(48, D * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(D, 48),
            nn.LeakyReLU(True),
            nn.Linear(48, 96),
            nn.LeakyReLU(True),
            nn.Linear(96, 22),
            nn.Sigmoid()
        )
        
        flow_init = RealNVP(dim=D)
        flows_init = [flow_init for _ in range(K)]
        prior = MultivariateNormal(torch.zeros(D).cuda(), torch.eye(D).cuda())
        self.flows = NormalizingFlowModel(prior, flows_init)

    def forward(self, x):
        # Run Encoder and get NF params
        enc = self.encoder(x)
        mu = enc[:, :self.dim]
        log_var = enc[:, self.dim: self.dim * 2]

        # Re-parametrize
        sigma = (log_var * .5).exp()
        z = mu + sigma * torch.randn_like(sigma)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        # Construct more expressive posterior with NF
        
        z_k, _, sum_ladj = self.flows(z)
        
        kl_div = kl_div / x.size(0) - sum_ladj.mean()  # mean over batch

        # Run Decoder
        x_prime = self.decoder(z_k)
        return x_prime, kl_div

## Creating Instance¶

In [7]:
N_EPOCHS = 20
PRINT_INTERVAL = 2000
NUM_WORKERS = 4
LR = 1e-4

N_FLOWS = 4
Z_DIM = 4

n_steps = 0

In [8]:
model = VAE_NF(N_FLOWS, Z_DIM).cuda()

In [9]:
optimizer = optim.Adam(model.parameters(), lr=LR)

In [10]:
def train():
    global n_steps
    train_loss = []
    model.train()

    for batch_idx, x in enumerate(bkgAE_train_iterator):
        start_time = time.time()
        
        x = x.float().cuda()

        x_tilde, kl_div = model(x)
        loss_recons = F.binary_cross_entropy(x_tilde, x, size_average=False) / x.size(0)
        loss = loss_recons + kl_div

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss.append([loss_recons.item(), kl_div.item()])

        if (batch_idx + 1) % PRINT_INTERVAL == 0:
            print('\tIter [{}/{} ({:.0f}%)]\tLoss: {} Time: {:5.3f} ms/batch'.format(
                batch_idx * len(x), 50000,
                PRINT_INTERVAL * batch_idx / 50000,
                np.asarray(train_loss)[-PRINT_INTERVAL:].mean(0),
                1000 * (time.time() - start_time)
            ))

        n_steps += 1

In [11]:
def evaluate(split='valid'):
    global n_steps
    start_time = time.time()
    val_loss = []
    model.eval()

    with torch.no_grad():
        for batch_idx, x in enumerate(bkgAE_test_iterator):
            
            x = x.float().cuda()

            x_tilde, kl_div = model(x)
            loss_recons = F.binary_cross_entropy(x_tilde, x, size_average=False) / x.size(0)
            loss = loss_recons + kl_div

            val_loss.append(loss.item())
            
    print('\nEvaluation Completed ({})!\tLoss: {:5.4f} Time: {:5.3f} s'.format(
        split,
        np.asarray(val_loss).mean(0),
        time.time() - start_time
    ))
    return np.asarray(val_loss).mean(0)

In [12]:
BEST_LOSS = 99999
LAST_SAVED = -1
for epoch in range(1, N_EPOCHS):
    print("Epoch {}:".format(epoch))
    train()
    cur_loss = evaluate()

    if cur_loss <= BEST_LOSS:
        BEST_LOSS = cur_loss
        LAST_SAVED = epoch
        print("Saving model!")
        torch.save(model.state_dict(), "/data/t3home000/spark/QUASAR/weights/bkg_vae_NF_NVP.h5")
    else:
        print("Not saving model! Last saved: {}".format(LAST_SAVED))

Epoch 1:


/home/spark/miniconda3/envs/myenv/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Evaluation Completed (valid)!	Loss: -406.9379 Time: 6.911 s
Saving model!
Epoch 2:

Evaluation Completed (valid)!	Loss: -395.7585 Time: 6.974 s
Not saving model! Last saved: 1
Epoch 3:

Evaluation Completed (valid)!	Loss: -395.8453 Time: 6.955 s
Not saving model! Last saved: 1
Epoch 4:

Evaluation Completed (valid)!	Loss: -392.4381 Time: 6.917 s
Not saving model! Last saved: 1
Epoch 5:

Evaluation Completed (valid)!	Loss: -392.1311 Time: 7.056 s
Not saving model! Last saved: 1
Epoch 6:

Evaluation Completed (valid)!	Loss: -393.9002 Time: 7.014 s
Not saving model! Last saved: 1
Epoch 7:

Evaluation Completed (valid)!	Loss: -390.4122 Time: 7.168 s
Not saving model! Last saved: 1
Epoch 8:

Evaluation Completed (valid)!	Loss: -384.8078 Time: 7.104 s
Not saving model! Last saved: 1
Epoch 9:

Evaluation Completed (valid)!	Loss: -389.6537 Time: 6.890 s
Not saving model! Last saved: 1
Epoch 10:

Evaluation Completed (valid)!	Loss: -379.5034 Time: 6.879 s
Not saving model! Last saved: 1
Epoch 

## Testing with the trained model

In [30]:
def get_loss(dt_in):

    dt_in[:,1] = (dt_in[:,1]-np.mean(dt_in[:,1]))/np.std(dt_in[:,1])
    dt_in[:,2] = (dt_in[:,2]-np.mean(dt_in[:,2]))/np.std(dt_in[:,2])
    dt_in[:,3] = (dt_in[:,3]-np.mean(dt_in[:,3]))/np.std(dt_in[:,3])
    dt_in[:,4] = (dt_in[:,4]-np.mean(dt_in[:,4]))/np.std(dt_in[:,4])
    dt_in[:,5] = (dt_in[:,5]-np.mean(dt_in[:,5]))/np.std(dt_in[:,5])
    dt_in[:,6] = (dt_in[:,6]-np.mean(dt_in[:,6]))/np.std(dt_in[:,6])

    dt_in[:,8] = (dt_in[:,8]-np.mean(dt_in[:,8]))/np.std(dt_in[:,8])
    dt_in[:,9] = (dt_in[:,9]-np.mean(dt_in[:,9]))/np.std(dt_in[:,9])
    dt_in[:,10] = (dt_in[:,10]-np.mean(dt_in[:,10]))/np.std(dt_in[:,10])
    dt_in[:,11] = (dt_in[:,11]-np.mean(dt_in[:,11]))/np.std(dt_in[:,11])
    dt_in[:,12] = (dt_in[:,12]-np.mean(dt_in[:,12]))/np.std(dt_in[:,12])

    dt_in[:,14] = (dt_in[:,14]-np.mean(dt_in[:,14]))/np.std(dt_in[:,14])
    dt_in[:,15] = (dt_in[:,15]-np.mean(dt_in[:,15]))/np.std(dt_in[:,15])
    dt_in[:,16] = (dt_in[:,16]-np.mean(dt_in[:,16]))/np.std(dt_in[:,16])
    dt_in[:,17] = (dt_in[:,17]-np.mean(dt_in[:,17]))/np.std(dt_in[:,17])
    dt_in[:,18] = (dt_in[:,18]-np.mean(dt_in[:,18]))/np.std(dt_in[:,18])
    dt_in[:,19] = (dt_in[:,19]-np.mean(dt_in[:,19]))/np.std(dt_in[:,19])
    
    dt_in[:,21] = (dt_in[:,21]-np.mean(dt_in[:,21]))/np.std(dt_in[:,21])
    dt_in[:,22] = (dt_in[:,22]-np.mean(dt_in[:,22]))/np.std(dt_in[:,22])
    dt_in[:,23] = (dt_in[:,23]-np.mean(dt_in[:,23]))/np.std(dt_in[:,23])
    dt_in[:,24] = (dt_in[:,24]-np.mean(dt_in[:,24]))/np.std(dt_in[:,24])
    dt_in[:,25] = (dt_in[:,25]-np.mean(dt_in[:,25]))/np.std(dt_in[:,25])   
    
    total_in = torch.tensor(dt_in)
    total_in_train_x_1 = total_in.t()[1:7].t()
    total_in_train_x_2 = total_in.t()[8:13].t()
    total_in_train_x_3 = total_in.t()[14:20].t()
    total_in_train_x_4 = total_in.t()[21:26].t()
    total_in_selection = torch.cat((total_in_train_x_1,total_in_train_x_2,total_in_train_x_3,total_in_train_x_4),dim=1)
    
    with torch.no_grad():
        loss_total_in = torch.mean((model(total_in_selection.float().cuda())[0]- total_in_selection.float().cuda())**2,dim=1).data.cpu().numpy()
    
    return loss_total_in

In [14]:
def get_mass(inputstring):

    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    
    return dt_in[:,0]

In [15]:
bb2mass = get_mass("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_BB1.h5")
purebkgmass = get_mass("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_pureBkg.h5")

FileNotFoundError: File ../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_BB1.h5 does not exist

In [ ]:
bb2mass, bb2mmdt1, bb2mmdt2, bb2prun1,bb2prun2, bb2loss = get_mass_and_loss("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_BB1.h5")
purebkgmass, purebkgmmdt1, purebkgmmdt2, purebkgprun1,purebkgprun2, purebkgloss = get_mass_and_loss("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_pureBkg.h5")

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
bins = np.linspace(0,5,1100)
plt.hist(bb2loss,bins=bins,alpha=0.3,color='b',label='blackbox1')
plt.hist(purebkgloss,bins=bins,alpha=0.3,color='r',label='background')
plt.xlabel(r'Autoencoder Loss')
plt.ylabel('Count')
plt.legend(loc='upper right')
plt.show()

In [23]:
f = pd.read_hdf("/data/t3home000/spark/LHCOlympics/data/MassRatio_RandD.h5")
dt = f.values

In [24]:
f.head()

,Mjj,Mj1,j1 tau21,j1 tau32,j1 tau43,j1 sqrt(tau^2_1)/tau^1_1,j1 n_trk,j1 pT1,j1 M_trim,j1 M_prun,...,j2 sqrt(tau^2_1)/tau^1_1,j2 n_trk,j2 pT1,j2 M_trim,j2 M_prun,j2 M_mmdt,j2 M_sdb1,j2 M_sdb2,j2 M_sdm1,isSignal
0,2577.571899,98.677270,0.528903,0.788281,0.904471,4.241889,136.0,1285.895950,18.881765,9.797733,...,1.895988,128.0,1282.286017,42.162664,18.466533,18.466533,31.845136,42.162664,0.000000,0.0
1,3807.507389,584.595432,0.345626,0.463461,0.865982,1.069972,320.0,1334.493332,556.665923,562.607897,...,1.377217,348.0,1306.137883,395.226881,393.309512,405.034096,405.034096,405.034096,405.034096,0.0
2,1710.965414,159.597526,0.677692,0.690707,0.695322,1.310040,332.0,678.557182,144.351550,142.366275,...,1.887494,236.0,1072.462085,54.235070,41.967840,41.352112,51.721630,70.442364,-0.000003,0.0
3,2603.379037,515.237299,0.091038,0.784454,0.860716,1.102743,248.0,1284.020224,501.564320,506.727622,...,1.997360,352.0,1217.031950,81.842001,60.307703,60.307703,72.423677,84.480859,0.000003,0.0
4,3294.162200,142.420213,0.507714,0.522686,0.904070,1.853319,220.0,1087.658980,129.146700,36.160229,...,1.113248,204.0,1205.343324,103.456059,99.817788,103.456059,103.456059,103.456059,0.000008,1.0


In [26]:
idx = dt[:,27]
bkg_idx = np.where(idx==0)[0]
signal_idx = np.where(idx==1)[0]

In [28]:
print(idx[4])

1.0


In [29]:
def get_mass_and_loss(inputstring):
    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    dt_in[:,1] = (dt_in[:,1]-np.mean(dt_in[:,1]))/np.std(dt_in[:,1])
    dt_in[:,2] = (dt_in[:,2]-np.mean(dt_in[:,2]))/np.std(dt_in[:,2])
    dt_in[:,3] = (dt_in[:,3]-np.mean(dt_in[:,3]))/np.std(dt_in[:,3])
    dt_in[:,4] = (dt_in[:,4]-np.mean(dt_in[:,4]))/np.std(dt_in[:,4])
    dt_in[:,5] = (dt_in[:,5]-np.mean(dt_in[:,5]))/np.std(dt_in[:,5])
    dt_in[:,6] = (dt_in[:,6]-np.mean(dt_in[:,6]))/np.std(dt_in[:,6])

    dt_in[:,8] = (dt_in[:,8]-np.mean(dt_in[:,8]))/np.std(dt_in[:,8])
    dt_in[:,9] = (dt_in[:,9]-np.mean(dt_in[:,9]))/np.std(dt_in[:,9])
    dt_in[:,10] = (dt_in[:,10]-np.mean(dt_in[:,10]))/np.std(dt_in[:,10])
    dt_in[:,11] = (dt_in[:,11]-np.mean(dt_in[:,11]))/np.std(dt_in[:,11])
    dt_in[:,12] = (dt_in[:,12]-np.mean(dt_in[:,12]))/np.std(dt_in[:,12])

    dt_in[:,14] = (dt_in[:,14]-np.mean(dt_in[:,14]))/np.std(dt_in[:,14])
    dt_in[:,15] = (dt_in[:,15]-np.mean(dt_in[:,15]))/np.std(dt_in[:,15])
    dt_in[:,16] = (dt_in[:,16]-np.mean(dt_in[:,16]))/np.std(dt_in[:,16])
    dt_in[:,17] = (dt_in[:,17]-np.mean(dt_in[:,17]))/np.std(dt_in[:,17])
    dt_in[:,18] = (dt_in[:,18]-np.mean(dt_in[:,18]))/np.std(dt_in[:,18])
    dt_in[:,19] = (dt_in[:,19]-np.mean(dt_in[:,19]))/np.std(dt_in[:,19])
    
    dt_in[:,21] = (dt_in[:,21]-np.mean(dt_in[:,21]))/np.std(dt_in[:,21])
    dt_in[:,22] = (dt_in[:,22]-np.mean(dt_in[:,22]))/np.std(dt_in[:,22])
    dt_in[:,23] = (dt_in[:,23]-np.mean(dt_in[:,23]))/np.std(dt_in[:,23])
    dt_in[:,24] = (dt_in[:,24]-np.mean(dt_in[:,24]))/np.std(dt_in[:,24])
    dt_in[:,25] = (dt_in[:,25]-np.mean(dt_in[:,25]))/np.std(dt_in[:,25])   
    
    total_in = torch.tensor(dt_in)
    total_in_train_x_1 = total_in.t()[1:7].t()
    total_in_train_x_2 = total_in.t()[8:13].t()
    total_in_train_x_3 = total_in.t()[14:20].t()
    total_in_train_x_4 = total_in.t()[21:26].t()
    total_in_selection = torch.cat((total_in_train_x_1,total_in_train_x_2,total_in_train_x_3,total_in_train_x_4),dim=1)
    
    with torch.no_grad():
        loss_total_in = torch.mean((model(total_in_selection.float().cuda())[0]- total_in_selection.float().cuda())**2,dim=1).data.cpu().numpy()
    
    f_in = pd.read_hdf(inputstring)
    dt_in = f_in.values
    
    return dt_in[:,0], dt_in[:,10], dt_in[:,23], dt_in[:,9], dt_in[:,22], loss_total_in

In [ ]:
bb2mass, bb2mmdt1, bb2mmdt2, bb2prun1,bb2prun2, bb2loss = get_mass_and_loss("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_BB1.h5")
purebkgmass, purebkgmmdt1, purebkgmmdt2, purebkgprun1,purebkgprun2, purebkgloss = get_mass_and_loss("../../../2_lhc/LHC_Olympics2020/processing/test_dataset/MassRatio_pureBkg.h5")

In [19]:
data_bkg = torch.tensor(dt[bkg_idx])
data_signal = torch.tensor(dt[signal_idx])

In [20]:
data_train_x_1 = data_bkg.t()[0:6].t()
data_train_x_2 = data_bkg.t()[7:13].t()
data_test_bkg = torch.cat((data_train_x_1,data_train_x_2),dim=1)

In [21]:
data_train_x_1 = data_signal.t()[0:6].t()
data_train_x_2 = data_signal.t()[7:13].t()
data_test_signal = torch.cat((data_train_x_1,data_train_x_2),dim=1)

In [31]:
loss_bkg = get_loss(dt[bkg_idx,:])

In [32]:
loss_sig = get_loss(dt[signal_idx,:])

In [33]:
def get_tpr_fpr(sigloss,bkgloss,aetype='sig'):
    bins = np.linspace(0,50,1001)
    tpr = []
    fpr = []
    for cut in bins:
        if aetype == 'sig':
            tpr.append(np.where(sigloss<cut)[0].shape[0]/len(sigloss))
            fpr.append(np.where(bkgloss<cut)[0].shape[0]/len(bkgloss))
        if aetype == 'bkg':
            tpr.append(np.where(sigloss>cut)[0].shape[0]/len(sigloss))
            fpr.append(np.where(bkgloss>cut)[0].shape[0]/len(bkgloss))
    return tpr,fpr        

In [34]:
bkg_tpr, bkg_fpr = get_tpr_fpr(loss_sig,loss_bkg,aetype='bkg')

In [36]:
np.save('NFLOWVAE_NVP_bkgAE_fpr.npy',bkg_fpr)
np.save('NFLOWVAE_NVP_bkgAE_tpr.npy',bkg_tpr)